In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
import os
os.chdir("gdrive/MyDrive/EE298 2022/REQ02")

In [3]:
import os
import torch
import torch.utils.data
import torchvision
from PIL import Image
from pycocotools.coco import COCO

class myOwnDataset(torch.utils.data.Dataset):
    def __init__(self, root, annotation, transforms=None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(annotation)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        # Own coco file
        coco = self.coco
        # Image ID
        img_id = self.ids[index]
        # List: get annotation id from coco
        ann_ids = coco.getAnnIds(imgIds=img_id)
        # Dictionary: target coco_annotation file for an image
        coco_annotation = coco.loadAnns(ann_ids)
        # path for input image
        path = coco.loadImgs(img_id)[0]['file_name']
        # open the input image
        img = Image.open(os.path.join(self.root, path))

        # number of objects in the image
        num_objs = len(coco_annotation)

        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        boxes = []
        labels=[]
        for i in range(num_objs):
            xmin = coco_annotation[i]['bbox'][0]
            ymin = coco_annotation[i]['bbox'][1]
            xmax = xmin + coco_annotation[i]['bbox'][2]
            ymax = ymin + coco_annotation[i]['bbox'][3]
            label= coco_annotation[i]['category_id']
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(label)
           
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels=torch.as_tensor(labels,dtype=torch.int64)
        # Labels (In my case, I only one class: target class or background)
        #labels=coco.getCatIds(imgIds=img_id)
        # Tensorise img_id
        img_id = torch.tensor([img_id])
        # Size of bbox (Rectangular)
        areas = []
        for i in range(num_objs):
            areas.append(coco_annotation[i]['area'])
        areas = torch.as_tensor(areas, dtype=torch.float32)
        # Iscrowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Annotation is in dictionary format
        my_annotation = {}
        my_annotation["boxes"] = boxes
        my_annotation["labels"] = labels
        my_annotation["image_id"] = img_id
        my_annotation["area"] = areas
        my_annotation["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, my_annotation

    def __len__(self):
        return len(self.ids)

In [4]:
from engine import train_one_epoch, evaluate
import utils


def get_transform():
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)

In [5]:
import utils
train_data_dir = r"/content/gdrive/MyDrive/EE298 2022/REQ02/Images"
test_data_dir=r"/content/gdrive/MyDrive/EE298 2022/REQ02/Images"
train_coco = r"/content/gdrive/MyDrive/EE298 2022/REQ02/train.json"
test_coco=r"/content/gdrive/MyDrive/EE298 2022/REQ02/test.json"
def collate_fn(batch):
    return tuple(zip(*batch))
# create own Dataset
my_dataset = myOwnDataset(root=train_data_dir,
                          annotation=train_coco,
                          transforms=get_transform()
                          )
my_dataset_test = myOwnDataset(root=test_data_dir,
                          annotation=test_coco,
                          transforms=get_transform()
                          )



# Batch size
train_batch_size = 8

# own DataLoader
data_loader = torch.utils.data.DataLoader(my_dataset,
                                          batch_size=train_batch_size,
                                          shuffle=True,
                                          num_workers=4,
                                          collate_fn=utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(my_dataset_test,
                                          batch_size=1,
                                          shuffle=False,
                                          num_workers=4,
                                          collate_fn=utils.collate_fn)

loading annotations into memory...
Done (t=0.64s)
creating index...
index created!
loading annotations into memory...
Done (t=0.63s)
creating index...
index created!


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [6]:
# select device (whether GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# DataLoader is iterable over Dataset
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[{'boxes': tensor([[411., 116., 521., 159.]], device='cuda:0'), 'labels': tensor([3], device='cuda:0'), 'image_id': tensor([1121], device='cuda:0'), 'area': tensor([4730.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}, {'boxes': tensor([[273., 320., 309., 408.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0'), 'image_id': tensor([1864], device='cuda:0'), 'area': tensor([3168.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}, {'boxes': tensor([[ 24., 126., 130., 384.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0'), 'image_id': tensor([989], device='cuda:0'), 'area': tensor([27348.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}, {'boxes': tensor([[252., 104., 328., 305.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0'), 'image_id': tensor([1689], device='cuda:0'), 'area': tensor([15276.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}, {'boxes': tensor([[313., 192., 387., 405.]], device='cuda:0'), 'labels

KeyboardInterrupt: ignored

In [7]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

def get_model_instance_segmentation_load(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model


In [12]:
def load_checkpoint(model, optimizer, load_path):
    checkpoint = torch.load(load_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    
    return model, optimizer, epoch
  
num_classes =4
num_epochs = 10
model = get_model_instance_segmentation(num_classes)
model.to(device)   
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=0.0005)

#len_dataloader = len(data_loader)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)


model,optimizer,epoch=load_checkpoint(model,optimizer,'trial_withtrain.pth')


In [13]:
for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: [0]  [  0/348]  eta: 0:53:54  lr: 0.000019  loss: 0.0654 (0.0654)  loss_classifier: 0.0459 (0.0459)  loss_box_reg: 0.0175 (0.0175)  loss_objectness: 0.0014 (0.0014)  loss_rpn_box_reg: 0.0006 (0.0006)  time: 9.2946  data: 3.7311  max mem: 9793


KeyboardInterrupt: ignored

In [ ]:
def save_checkpoint(model, optimizer, save_path, epoch):
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch
    }, save_path)


save_checkpoint(model,optimizer,"Model.pth",epoch)